In [1]:
import request_category as rc
import database as db
from download import download

import pickle
from pathlib import Path
import re

import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## Machine learning - Grab all the unique articles with title and pageid

inner_ml = """SELECT stuff.category, stuff.subcategory, pc.pageid 
              FROM ( SELECT category, subcategory, subcategory_id 
                     FROM subcategories sc 
                         JOIN categories c 
                         ON sc.category_id = c.category_id 
                     WHERE category ='machine learning' ) as stuff 
              JOIN page_category pc 
              ON stuff.subcategory_id = pc.subcategory_id             
              """  #               HAVING COUNT(pc.pageid) = 1 ,        

ml_pages_query = """SELECT crap_ml.category, crap_ml.subcategory, p.pageid, p.title, p.article
                    FROM ({}) as crap_ml
                    JOIN pages p
                    ON crap_ml.pageid = p.pageid
                    
                """.format( inner_ml) # LIMIT 1000  ## GROUP BY crap_ml.category, crap_ml.subcategory, p.pageid
                    #HAVING COUNT( crap_ml.pageid) = 1, DISTINCT,  crap_ml.subcategory,
ml_pages_query = re.sub( "\s+"," ", ml_pages_query)
ml_pages_query


"SELECT crap_ml.category, crap_ml.subcategory, p.pageid, p.title, p.article FROM (SELECT stuff.category, stuff.subcategory, pc.pageid FROM ( SELECT category, subcategory, subcategory_id FROM subcategories sc JOIN categories c ON sc.category_id = c.category_id WHERE category ='machine learning' ) as stuff JOIN page_category pc ON stuff.subcategory_id = pc.subcategory_id ) as crap_ml JOIN pages p ON crap_ml.pageid = p.pageid "

In [3]:
## Business Software - Grab all the unique articles with title and pageid
#inner_bs = db.category_query( ('\'business software\''))
inner_bs = """SELECT stuff.category, stuff.subcategory, pc.pageid 
                        FROM ( SELECT category, subcategory, subcategory_id 
                               FROM subcategories sc 
                                   JOIN categories c 
                                   ON sc.category_id = c.category_id 
                               WHERE category ='business software' ) as stuff 
                        JOIN page_category pc 
                        ON stuff.subcategory_id = pc.subcategory_id
                        """   #  GROUP BY stuff.category, stuff.subcategory, pc.pageid

bs_pages_query = """SELECT crap_bs.category, crap_bs.subcategory, p.pageid, p.title, p.article
                    FROM ({}) as crap_bs
                    JOIN pages p
                    ON crap_bs.pageid = p.pageid
                    
                 """.format(inner_bs)  ## DISTINCT, LIMIT 1000, HAVING COUNT(crap_bs.pageid) = 1, crap_bs.category, crap_bs.subcategory,
bs_pages_query = re.sub( "\s+"," ", bs_pages_query)
bs_pages_query

"SELECT crap_bs.category, crap_bs.subcategory, p.pageid, p.title, p.article FROM (SELECT stuff.category, stuff.subcategory, pc.pageid FROM ( SELECT category, subcategory, subcategory_id FROM subcategories sc JOIN categories c ON sc.category_id = c.category_id WHERE category ='business software' ) as stuff JOIN page_category pc ON stuff.subcategory_id = pc.subcategory_id ) as crap_bs JOIN pages p ON crap_bs.pageid = p.pageid "

In [4]:
pages_query = """SELECT b.category, b.subcategory, b.title, b.pageid, b.article
                 FROM (({}) UNION ({}) ) as b;""".format( ml_pages_query, bs_pages_query)

pages_df = db.query_to_dataframe( pages_query)
pages_df.shape  # shape = (2425, 3) Before adding category and subcategory
                ## shape = (2768, 3) after adding Distinct category and subcategory in outer query


(2768, 5)

In [2]:
from search import get_data

In [3]:
pages_df = get_data('machine learning', 'business software')
pages_df.shape

(2756, 4)

In [ ]:

pages_df.head()

In [ ]:
#print( 'Total titles: {}'.format(pages_df.drop( ['pageid'], axis = 1).groupby( by = ['category', 'subcategory']).count().sum() ) )
pages_df.drop( ['pageid'], axis = 1).groupby( by = ['category', 'subcategory']).count()





In [ ]:
pages_df[ pages_df.pageid == 26137900]  # 11273721

In [ ]:
test = pages_df.drop_duplicates( subset = ['category', 'pageid','title'], keep='last').copy()  ## 'category', 

In [ ]:
duplicates_df = test[( test.pageid == 12185719) | (test.pageid == 41732818 ) | ( test.pageid == 462546) | (test.pageid == 36089423)]  # 11273721

duplicates_df.drop( ['pageid'], axis = 1).groupby( by = ['category', 'subcategory']).count()





In [ ]:
#print( 'Total titles: {}'.format(pages_df.drop( ['pageid'], axis = 1).groupby( by = ['category', 'subcategory']).count().sum() ) )
#pages_df.drop( ['pageid'], axis = 1).groupby( by = ['category', 'subcategory']).count()

test.drop( ['pageid'], axis = 1).groupby( by = ['category', 'subcategory']).count()



In [ ]:
pages_df.pageid == value_counts()

In [ ]:

test.pageid.value_counts()

## As is, no removal of duplicates

## MNLogistic,  Build classifier for ML or BS

In [ ]:
pages_df.columns

In [5]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelBinarizer, LabelEncoder

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
import sklearn.externals.joblib

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
X = pages_df['article'].copy()
#pages_df[['category']]

#X.index = X.pageid
#X.drop(['pageid'], axis = 1, inplace = True)

#Y = pages_df[['category', 'subcategory']].copy()

#lb = LabelBinarizer()

encoder = LabelEncoder()

y = encoder.fit_transform( pages_df['category'] )

In [8]:
X_train, X_test, y_train, y_test = train_test_split( X, y, random_state = 42)

In [41]:
y_train

array([1, 0, 0, ..., 0, 0, 0])

In [42]:
X_train

1584    encog machine learning framework available jav...
1800    ibm current microsoft windows based graphical ...
2429    mtconnect manufacturing technical standard ret...
1706    quake markup language quakeml flexible extensi...
1027    micai short mexican international conference a...
240     spider project project management software dev...
1032    fasttrack schedule project management software...
582     waffles collection command line tools performi...
1792    highdeal privately owned bss software product ...
862     procurify cloud based procurement software com...
1681    webengage privately held saas company based mu...
1435    xgboost open source software library provides ...
350     statistical learning theory representer theore...
2537    balanced scorecard bsc strategy performance ma...
1058    deeplearningj deep learning programming librar...
1966    geoffrey everest hinton frs born december brit...
1491    easynote web based project management tool cre...
1547    solvei

In [9]:
vectorizer = TfidfVectorizer( ngram_range = (1,2),
                             min_df = 3, max_df = .9, 
                             stop_words = 'english')
                             
                             

In [10]:
X_train_sps = vectorizer.fit_transform( X_train) # X_train['article']

X_test_sps = vectorizer.transform( X_test)

train_prior_df = pd.DataFrame(  X_train_sps.toarray(), 
                     columns = vectorizer.get_feature_names())


test_prior_df = pd.DataFrame( X_test_sps.toarray(),
                            columns = vectorizer.get_feature_names())

In [ ]:
y_train#.reshape(-1,1)

In [ ]:
train_prior_df.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
X_

In [26]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit

import sklearn.externals.joblib

In [12]:
n_components = 700
SVD = TruncatedSVD(n_components)
component_names = ["component_"+str(i+1) for i in range(n_components)]


In [13]:
## dtm
train_svd_matrix = SVD.fit_transform( train_prior_df)

test_svd_matrix = SVD.transform( test_prior_df)
train_svd_matrix.shape


(2076, 700)

In [14]:
train_latent_semantic_analysis = pd.DataFrame(train_svd_matrix,
                                        index=train_prior_df.index,#nonempty_pages_df.pageid,
                                        columns=component_names)

test_latent_semantic_analysis = pd.DataFrame(test_svd_matrix,
                                        index=test_prior_df.index,#nonempty_pages_df.pageid,
                                        columns=component_names)

In [ ]:
latent_semantic_analysis.shape

In [15]:
logit = LogisticRegression()

In [16]:
logit.fit_transform( train_latent_semantic_analysis, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


array([[ 0.19645436, -0.00302654,  0.10013735, ..., -0.00348833,
         0.01514014,  0.01272561],
       [ 0.16481159, -0.15997568, -0.0382075 , ...,  0.0223714 ,
        -0.01490773, -0.00627058],
       [ 0.19434109, -0.14735876, -0.00815371, ...,  0.04600086,
        -0.00994328, -0.00448326],
       ..., 
       [ 0.09754335, -0.11228189, -0.01451931, ...,  0.0201884 ,
        -0.0216765 ,  0.02384504],
       [ 0.055216  , -0.06603691, -0.01101332, ..., -0.01660244,
        -0.00866171,  0.01749349],
       [ 0.09149554, -0.11063171, -0.04679369, ..., -0.03525711,
         0.03197779, -0.03531844]])

In [18]:
logit.score( train_latent_semantic_analysis, y_train)

0.97880539499036612

In [21]:
logit.score( test_latent_semantic_analysis, y_test )

0.95809248554913296

In [ ]:
vectorizer = CountVectorizer()
X_text     = vectorizer.fit_transform(df['text'])
prior      = pd.DataFrame(
    X_text.toarray(), 
    columns=vectorizer.get_feature_names()
)

prior.index = df['label']
prior

In [ ]:
X = pages_df[['article']].copy()
pages_df[['category']]

## As is, no removal of duplicates

## MNLogistic,  Build classifier for ML or BS

In [ ]:
#y.category.value_counts()

In [ ]:
X = pages_df['article'].copy()

encoder = LabelEncoder()
y = encoder.fit_transform( pages_df['category'] )
X_train, X_test, y_train, y_test = train_test_split( X, y, random_state = 42)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, random_state = 42)

In [30]:
np.logspace(-3,3,7)

array([  1.00000000e-03,   1.00000000e-02,   1.00000000e-01,
         1.00000000e+00,   1.00000000e+01,   1.00000000e+02,
         1.00000000e+03])

In [34]:
lr_pipe = Pipeline([
    ('encoder', TfidfVectorizer(ngram_range = (1,2),
                             min_df = 3, max_df = .9, 
                             stop_words = 'english')),
    ('truncator',TruncatedSVD(n_components=700) ),
    ('model', LogisticRegression())
])

lr_params = {
    'model__C':[1,10]  #np.logspace(-3,3,7)
}

gs_lr_pipe = GridSearchCV(lr_pipe, param_grid=lr_params, cv=5) # StratifiedShuffleSplit(n_splits=5)

gs_lr_pipe.fit(X_train,y_train)

gs_lr_pipe.best_score_

0.97495183044315992

In [4]:
import sklearn.externals.joblib

In [71]:
sklearn.externals.joblib.dump(gs_lr_pipe, './LogitModel1.p')

['./LogitModel1.p']

In [ ]:
gssklearn.externals.joblib.load('./LogitModel1.p')
#dump(gs_lr_pipe, './LogitModel1.p')

In [48]:
gs_lr_pipe.score( X_train, y_train)

0.99470134874759153

In [49]:
gs_lr_pipe.score( X_test, y_test)

0.97109826589595372

In [67]:
def get_json( search_term, query):
    r = requests.get( query)  ## request HTTP results
    response = r.json()
    pageinfo = response['query']['pages']
    if isinstance( search_term, str):
        
        pageid = list(pageinfo.keys())[0]
        text = pageinfo[pageid]['extract']
    elif isinstance( search_term, int):
        pageid = str(search_term)
        text = pageinfo[pageid]['extract']
        
    return text, pageid
    

def get_article( search_term ):# title or page id, Capitalization ignored
    base_url = 'https://en.wikipedia.org/w/api.php'
    if isinstance( search_term, str):
        #search_term = rc.format_search( search_term)
        tag = 'titles={}'.format( search_term)
    elif isinstance( search_term, int):
        tag = 'pageids={}'.format( search_term)
    else:
        print( 'Invalid search term...')
        return ''

    action_tag = "?action=query&prop=extracts&explaintext&{}&format=json".format( tag) 
    query = base_url + action_tag
    
    article_text, pageid = get_json( search_term, query)
    
    return article_text, pageid

In [60]:
import requests

In [64]:
article_text, pageid = get_article( 'Saffron Technology')
article_te

'https://en.wikipedia.org/w/api.php?action=query&prop=extracts&explaintext&titles=Saffron Technology&format=json'

In [6]:
search_doc, pageid = rc.get_article('Brain')  # Saffron Technology
search_doc = rc.cleaner( search_doc)
##print( pageid )
search_doc[0:200]

'brain organ serves center nervous system vertebrate invertebrate animals brain located head usually close sensory organs senses vision brain complex organ vertebrate s body human cerebral cortex conta'

In [37]:
gs_lr_pipe.transform([search_doc])

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


array([[  1.35170158e-01,  -3.80362820e-02,   5.87758322e-02,
          1.26285511e-01,  -2.40289597e-02,   1.81071763e-02,
          6.01996672e-02,   4.52544696e-02,  -3.72924130e-02,
          5.53361822e-03,   1.13731421e-03,   2.89964043e-02,
          2.26841906e-02,   1.79729990e-03,   1.33493254e-02,
          3.46916703e-03,  -1.38465959e-02,  -3.70860993e-03,
         -2.09624619e-02,  -3.69160943e-03,   5.82900874e-03,
          1.29680398e-02,  -2.58796948e-02,   1.32402450e-02,
          7.90851768e-02,   8.71380932e-03,  -1.85801961e-02,
          2.00458108e-02,  -1.88684068e-02,   3.19256858e-02,
          1.36056457e-02,   3.26715774e-02,   3.39892612e-05,
          4.75276091e-04,   3.92622375e-02,   2.36597557e-02,
          1.72693944e-02,  -2.54538977e-02,   2.81257282e-02,
          1.65605267e-02,  -1.57573774e-02,  -1.93420508e-02,
         -1.07358920e-02,  -3.39645447e-02,  -2.55434794e-02,
          6.01422475e-03,  -2.53142937e-02,   7.97866670e-02,
        

In [7]:
gs_lr_pipe.predict_proba([search_doc])  ## 'Brain'

NameError: name 'gs_lr_pipe' is not defined

In [47]:
gs_lr_pipe.predict_proba([search_doc])  ## 'Microsoft'

array([[ 0.99873895,  0.00126105]])

In [51]:
gs_lr_pipe.predict_proba([search_doc])  ## 'Statistics'

array([[ 0.00227817,  0.99772183]])

In [69]:
gs_lr_pipe.predict_proba([search_doc])  ## 'Saffron Technology'

array([[ 0.60603483,  0.39396517]])

In [ ]:
X = pages_df['article'].copy()

encoder = LabelEncoder()
y = encoder.fit_transform( pages_df['category'] )
X_train, X_test, y_train, y_test = train_test_split( X, y, random_state = 42)

In [ ]:
lr_pipe = Pipeline([
    ('encoder', TfidfVectorizer(ngram_range = (1,2),
                             min_df = 3, max_df = .9, 
                             stop_words = 'english')),
    ('truncator',TruncatedSVD(n_components=700) ),
    ('model', LogisticRegression())
])

lr_params = {
    'model__C':[1,10]  #np.logspace(-3,3,7)
}

gs_lr_pipe = GridSearchCV(lr_pipe, param_grid=lr_params, cv=5) # StratifiedShuffleSplit(n_splits=5)

gs_lr_pipe.fit(X_train,y_train)

gs_lr_pipe.best_score_